In [1]:
print("hello World!")

hello World!


In [2]:
%pwd

'/workspaces/End-to-End-Medical-Chatbot-AI-2-/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-Medical-Chatbot-AI-2-'

In [4]:
import sys, pkgutil
print(sys.executable)
print("langchain" in {m.name for m in pkgutil.iter_modules()})


/opt/conda/envs/medbot/bin/python
False


In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


/opt/conda/envs/medbot/bin/python


In [2]:
# Extarction of data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()

    return documents

In [ ]:
loader = DirectoryLoader("/full/path/to/Data", glob="*.pdf")
extracted_data = load_pdf_file(data = 'Data/')

FileNotFoundError: Directory not found: 'Data/'

In [7]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Path to Data folder
data_dir = os.path.join(os.getcwd(), "Data")

# Verify directory exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Directory not found: {data_dir}")

# Get all PDF files
pdf_files = [f for f in os.listdir(data_dir) if f.lower().endswith(".pdf")]

if not pdf_files:
    raise FileNotFoundError(f"No PDF files found in {data_dir}")

# Load each PDF separately
docs = []
for pdf_file in pdf_files:
    file_path = os.path.join(data_dir, pdf_file)
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())

print(f"Loaded {len(docs)} document chunks from {len(pdf_files)} PDF file(s).")



FileNotFoundError: Directory not found: /workspaces/End-to-End-Medical-Chatbot-AI-2-/research/Data